In [1]:
import pandas as pd

In [2]:
# Carrega o dataset
#df = pd.read_csv("dados/teste2.csv", encoding = 'utf-8')
df = pd.read_csv("dados/Dataset_Covid_CE.csv", encoding = 'utf-8')

/Users/paulocordeiro/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,5,6,7,8,9,10,11,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,47,49,50,51,52,53,54,55,56,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
todasColunas = df.columns.to_list()

In [4]:
listaCampos = ["dataNascimento", "sexoCaso", "racaCor", "municipioCaso","comorbidadeAsmaSivep","comorbidadeCardiovascularSivep","resultadoFinalExame"]

In [5]:
def criaIdentificador(row,ListaCampos):
    strID = ""
    for x in ListaCampos:
        strID = strID + str(row [x])
    return strID

In [6]:
def criaContador(df,campo):
    dfInterno = df[campo].value_counts().to_frame('Contador')
    dfInterno[campo] = dfInterno.index
    strQtd = 'qtd-' + campo

    try:
        df = df.drop(columns=[strQtd])
    except:
        pass
    
    dfInterno.columns = [strQtd,campo]
    return df.merge(dfInterno, left_on=campo, right_on=campo)

In [7]:
def suprimeCampo(row,nomeCampo,k):
    retorno = row[nomeCampo]
    if row['qtd-id'] < k:
        retorno = '*'
        
    return retorno

In [8]:
def retornaKAnom (df,strBlank):
    df1 = df[df["id"]!=strBlank]
    dfInterno = df1['qtd-id'].value_counts().to_frame('Contador')
    dfInterno['qtd-id'] = dfInterno.index
    return min(dfInterno['qtd-id'])

In [9]:
def anonimizadfk(df,colunasRemovidas,k):
    for x in listaCampos:
        df = criaContador(df,'id')
        dfk = retornaKAnom (df,strBlank)
        if dfk<=k:
            #print(x,dfk)
            df[x] = df.apply(suprimeCampo, axis=1,args=(x,k,))
            df['id'] = df.apply(criaIdentificador, axis=1,args=(listaCampos,))
            df = criaContador(df,'id')
        else:
            break

    dffinal = df.drop(columns=colunasRemovidas)
    strFile = 'Dataset_Covid_CE_' + str(k) + 'K.csv'
    dffinal.to_csv(strFile,index=False)

In [10]:
%%time
df['id'] = df.apply(criaIdentificador, axis=1,args=(listaCampos,))

CPU times: user 3min 1s, sys: 1.19 s, total: 3min 3s
Wall time: 3min 3s


In [11]:
%%time
df = criaContador(df,'id')

CPU times: user 11.5 s, sys: 1.58 s, total: 13 s
Wall time: 13 s


In [13]:
klist = [2,4,8,16]
strBlank = len(listaCampos)*'*'
colunasRemovidas = [item for item in todasColunas if item not in listaCampos]

In [14]:
%%time
for k in klist:
    print('Anonimizando dataset : ',k,"k")
    anonimizadfk(df,colunasRemovidas,k)

Anonimizando dataset :  2 k
Anonimizando dataset :  4 k
Anonimizando dataset :  8 k
Anonimizando dataset :  16 k
CPU times: user 2h 6min, sys: 3min 7s, total: 2h 9min 8s
Wall time: 2h 9min 10s
